<a href="https://colab.research.google.com/github/pradeep448/deep-learning/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 6 Complete [00h 01m 22s]
val_accuracy: 0.9116666913032532

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 00h 06m 08s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        134448    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 112)               2602096   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2,738,794
Trainable params: 2,738,794
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.1456 - accuracy: 0.9443 - val_loss: 0.2674 - val_accuracy: 0.9105
Epoch 5/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.1044 - accuracy: 0.9609 - val_loss: 0.3058 - val_accuracy: 0.9165
Epoch 6/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0762 - accuracy: 0.9715 - val_loss: 0.3348 - val_accuracy: 0.9118
Epoch 7/10
1688/1688 [==============================] - 20s 12ms/step - loss: 0.0579 - accuracy: 0.9788 - val_loss: 0.3621 - val_accuracy: 0.9187
Epoch 8/10
1688/1688 [==============================] - 19s 12ms/step - loss: 0.0438 - accuracy: 0.9840 - val_loss: 0.4118 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0332 - accuracy: 0.9887 - val_loss: 0.4529 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 19s 12ms/step - loss: 0.0283 - accuracy: 0.9895 - val_loss: 0.5023 